# Leitura do banco de dados

In [1]:
import pandas as pd

# parameters
dataset = "amostra.xlsx" #@param {type:"string"}
group_col = "" #@param {type:"string",label:"coluna para o agrupamento"}
target = "" # pode ter ou não


def decimal_converter(value):
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return value
    
numerical_cols = [
    'Tempo de Dispensa (dias)', 
    'Tempo de Entrega',
]

features = [
    'Area de atuacao [Funcionario]',
    'Descricao do afastamento',
]

converter_dict = {col: decimal_converter for col in numerical_cols}

df = pd.read_excel(dataset, index_col=None, header=0, converters=converter_dict)
df_num = df[numerical_cols]

df = df_num.join(df[features]).dropna(how='any')

target = 'Area de atuacao [Funcionario]'

# df = pd.read_csv('Iris.csv')
# target = 'variety'


df.head()



,Tempo de Dispensa (dias),Tempo de Entrega,Area de atuacao [Funcionario],Descricao do afastamento
0,3.1,1.0,Estagiário,"Sintomas, sinais e achados anormais"
1,3.6,6.0,Mecânico Manutenção III,Infecções virais de pele e de mucosas
2,10.1,4.0,Inspetor Qualidade III,Fatores que influenciam estado de saúde
3,13.0,11.0,Inspetor Qualidade III,Infecções virais de pele e de mucosas
4,4.3,0.0,Inspetor Qualidade III,"Sintomas, sinais e achados anormais"


# Análise de Outliers

In [2]:
from insights.analysis.outliers_analysis_no_schema import outlier_analysis

text, figures, filtered_df, title, numerical_describer, categorical_describer, outliers_df = outlier_analysis(df)

print(text)
for figure in figures:
    figure.show()

Utilizando o método DBscan foram detectados 57 outliers neste dataset, correspondendo a uma proporção de 6.71% do conjunto de amostras.
Utilizando o método Isolation Forest foram detectados 43 outliers neste dataset, correspondendo a uma proporção de 5.06% do conjunto de amostras.



In [3]:
from IPython.display import display
print(f'Análise dos outliers')
for t, n_des, c_des in zip(title, numerical_describer, categorical_describer):
    print(f'\tMétodo: {t}')
    print('\tFeatures numéricas')
    display(n_des)
    print('\tFeatures Categóricas')
    display(c_des)

Análise dos outliers
	Método: DBscan
	Features numéricas


,Tempo de Dispensa (dias),Tempo de Entrega
count,57.000000,57.000000
mean,20.138596,28.280702
std,32.788084,35.796426
min,0.000000,-14.000000
25%,1.200000,13.000000
50%,5.800000,17.000000
75%,13.400000,24.000000
max,165.900000,194.000000


	Features Categóricas


,Descricao do afastamento,Area de atuacao [Funcionario]
count,57,57
unique,13,18
top,Infecções virais de pele e de mucosas,Operador Manufatura I
freq,11,19


	Método: Isolation Forest
	Features numéricas


,Tempo de Dispensa (dias),Tempo de Entrega
count,43.000000,43.000000
mean,26.604651,32.744186
std,35.743251,40.199558
min,0.000000,-14.000000
25%,1.300000,9.500000
50%,13.000000,20.000000
75%,44.100000,49.500000
max,165.900000,194.000000


	Features Categóricas


,Descricao do afastamento,Area de atuacao [Funcionario]
count,43,43
unique,11,17
top,Doenças do sistema osteomuscular,Operador Manufatura I
freq,9,15


# Insights de Outliers

In [9]:
from insights.outliers.insights import outlier_insights
from insights.analysis.cluster_analysis_no_schema import cluster_analysis, numerical_insights, categorical_insights

numerical_group, categorical_group = outlier_insights(outliers_df, df, features)

text, tables = categorical_insights(categorical_group)
print(text)
display(tables[0])

É possível observar nas tabelas abaixo alguns dados notáveis extraídos dos grupos da base de dados. Estas tabelas de diferenças máximas e mínimas visam demonstrar as diferenças de distribuição entre a população e os grupos. A linha Contagem mostra a contagem daquela classe dentro do grupo, A linha Proporção mostra a proporção da classe em relação a população do grupo, A linha Diferença da População mostra a diferença de proporção daquela população no grupo em relação a população geral. 


,Area de atuacao [Funcionario],Descricao do afastamento
maior ocorrencia,Operador Manufatura I,Doenças do sistema osteomuscular
contagem,15,9
proporção,0.348837,0.209302
diferença da população,-0.033966,0.093872
grupo,0,0


In [10]:
texts, figures, tables = numerical_insights(numerical_group)

print(texts[0])
display(tables[0])
print(texts[1])
display(tables[1])
print(texts[2])
for figure in figures:
    figure.show()
print(texts[3])

Insights obtidos das variáveis numéricas estão disponíveis nas tabelas a seguir, onde são apresentadas as diferenças das médias e variâncias entre a população geral e cada um dos grupos. A idéia é facilitar a observação de tendências distintas em cada um dos grupos, em relação a população geral. A tabela abaixo é relacionada com a diferença de médias entre os grupos e a população geral.


cluster_group,0
Tempo de Dispensa (dias),21.945758
Tempo de Entrega,28.091654


Abaixo podemos observar a tabela de variância entre os grupos e a população geral. Esta tabela de variância é importante de um ponto de vista de análise da variação das features dentro de cada um dos grupos. A ideia é que a variância dentro de um grupo específico seja menor em relação a população em geral.


cluster_group,0
Tempo de Dispensa (dias),25.787145
Tempo de Entrega,28.750768


Figuras também são aliadas importantes na visualização de dados. Nas figuras abaixo estão presentes duas figuras que representam a variação de média e variância de cada grupo em relação a população geral. Dados variando para a cor azul significam que a variação é negativa, enquanto dados variando para cores vermelhas significam que a variação é positiva.



A maior diferença populacional positiva foi detectada na feature Tempo de Entrega e no grupo 0, com valor de 28.09. A maior variação negativa foi na feature Tempo de Dispensa (dias) e no grupo 0, com o valor registrado de 21.95



# Análise de Cluster

In [6]:
from insights.analysis.cluster_analysis_no_schema import cluster_analysis, numerical_insights, categorical_insights

texts, tables, numerical_group, categorical_group = cluster_analysis(filtered_df, target_variable = target, dimensions=2, categorical=False)

for text, table in zip(texts, tables):
    print(text)
    display(table)
    


A melhor separação de grupos ocorreu nas features: ['Tempo de Dispensa (dias)', 'Tempo de Entrega'] com uma quantidade de 10 grupos. A análise multidimensional em 2 dimensões fornece insights de quais features são mais importantes e que distinguem grupos entre si. Esta análise é realizada realizando a permutação das features do dataset.


,0,1,2,3,4,5,6,7,8,9
Operador Manufatura I,0.164516,0.041935,0.032258,0.070968,0.041935,0.187097,0.048387,0.219355,0.116129,0.077419
Controlador Produção,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
Facilitador,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.500000,0.000000,0.000000
Conferente Almoxarifado,0.000000,0.250000,0.250000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
Técnico Mecânico,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000
Mecânico Manutenção III,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.166667,0.333333
Analista Contábil Pl,0.200000,0.000000,0.000000,0.200000,0.000000,0.400000,0.000000,0.200000,0.000000,0.000000
Operador Empilhadeira,0.102041,0.020408,0.061224,0.040816,0.020408,0.265306,0.040816,0.244898,0.102041,0.102041
Analista Processos Jr,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000
Operador Manufatura III,0.208333,0.000000,0.083333,0.125000,0.083333,0.083333,0.041667,0.166667,0.125000,0.083333


A melhor separação de grupos ocorreu nas features: ['Tempo de Dispensa (dias)', 'Tempo de Entrega'] com uma quantidade de 8 grupos. A análise multidimensional de clusters é semelhante a análise bi-dimensional ou tri-dimensional, mas sua principal diferença é que utiliza todas as features disponíveis no dataset.


,0,1,2,3,4,5,6,7
Operador Manufatura I,0.100000,0.225806,0.077419,0.041935,0.054839,0.193548,0.232258,0.074194
Controlador Produção,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
Facilitador,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.500000,0.250000
Conferente Almoxarifado,0.000000,0.500000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000
Técnico Mecânico,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
Mecânico Manutenção III,0.000000,0.000000,0.333333,0.000000,0.166667,0.333333,0.166667,0.000000
Analista Contábil Pl,0.000000,0.400000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000
Operador Empilhadeira,0.122449,0.265306,0.061224,0.081633,0.040816,0.142857,0.244898,0.040816
Analista Processos Jr,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333
Operador Manufatura III,0.166667,0.083333,0.000000,0.083333,0.083333,0.291667,0.166667,0.125000


Na tabela abaixo podemos observar a comparação de todos os métodos de clustering testados. O método com o maior score será considerado o melhor método, sendo este utilizado nas próximas análises.


,Feature Permutation 2 dim.,Multidimensional
Tempo de Dispensa (dias) - Tempo de Entrega,0.422384,0.41016


In [7]:
texts, figures, tables = numerical_insights(numerical_group)

print(texts[0])
display(tables[0])
print(texts[1])
display(tables[1])
print(texts[2])
for figure in figures:
    figure.show()
print(texts[3])

Insights obtidos das variáveis numéricas estão disponíveis nas tabelas a seguir, onde são apresentadas as diferenças das médias e variâncias entre a população geral e cada um dos grupos. A idéia é facilitar a observação de tendências distintas em cada um dos grupos, em relação a população geral. A tabela abaixo é relacionada com a diferença de médias entre os grupos e a população geral.


cluster_group,0,1,2,3,4,5,6,7
Tempo de Dispensa (dias),3.561911,-2.049200,2.096911,-0.218645,9.894133,0.486911,-2.370381,-1.726271
Tempo de Entrega,-1.692308,0.657265,2.479487,9.846154,-0.776068,-1.947596,-1.971554,4.282517


Abaixo podemos observar a tabela de variância entre os grupos e a população geral. Esta tabela de variância é importante de um ponto de vista de análise da variação das features dentro de cada um dos grupos. A ideia é que a variância dentro de um grupo específico seja menor em relação a população em geral.


cluster_group,0,1,2,3,4,5,6,7
Tempo de Dispensa (dias),-2.056766,-2.487770,-1.620821,-0.854538,-1.350507,-2.483985,-2.732893,-2.179224
Tempo de Entrega,-1.540954,-2.316206,-1.588825,-1.041796,-1.126830,-2.055699,-2.282962,-1.782301


Figuras também são aliadas importantes na visualização de dados. Nas figuras abaixo estão presentes duas figuras que representam a variação de média e variância de cada grupo em relação a população geral. Dados variando para a cor azul significam que a variação é negativa, enquanto dados variando para cores vermelhas significam que a variação é positiva.



A maior diferença populacional positiva foi detectada na feature Tempo de Dispensa (dias) e no grupo 4, com valor de 9.89. A maior variação negativa foi na feature Tempo de Dispensa (dias) e no grupo 6, com o valor registrado de -2.37



In [8]:
text, tables = categorical_insights(categorical_group)
print(text)
print('\t Máximo')
display(tables[0])
print('\t Mínimo')
display(tables[1])

É possível observar nas tabelas abaixo alguns dados notáveis extraídos dos grupos da base de dados. Estas tabelas de diferenças máximas e mínimas visam demonstrar as diferenças de distribuição entre a população e os grupos. A linha Contagem mostra a contagem daquela classe dentro do grupo, A linha Proporção mostra a proporção da classe em relação a população do grupo, A linha Diferença da População mostra a diferença de proporção daquela população no grupo em relação a população geral. 
	 Máximo


,Descricao do afastamento,Area de atuacao [Funcionario]
maior ocorrencia,Doenças do sistema osteomuscular,Operador Manufatura I
contagem,15,23
proporção,0.333333,0.418182
diferença da população,0.222911,0.033566
grupo,4,7


	 Mínimo


,Descricao do afastamento,Area de atuacao [Funcionario]
maior ocorrencia,Fatores que influenciam estado de saúde,Operador Manufatura I
contagem,8,13
proporção,0.222222,0.361111
diferença da população,-0.042046,-0.023504
grupo,3,3
